In [1]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [3]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_03_fe_v02.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_03_fe_v02.parquet...
- [1 files][  6.9 GiB/  6.9 GiB]                                                
Operation completed over 1 objects/6.9 GiB.                                      


In [ ]:
base_path = '/home/eanegrin/'
dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_03_fe_v02.parquet'

semillas = [122219, 109279, 400391, 401537, 999961]

In [ ]:
data = pd.read_parquet(dataset_path + dataset_file)

In [5]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [ ]:
meses_excluidos = [201901,201902,201903,201904,
                   202108, 202109] # meses con clase ternaria incompleta

data = data[~data['foto_mes'].isin(meses_excluidos)]
data['foto_mes'].unique()

In [8]:
data.shape

(4066168, 682)

In [9]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,201905,0.994830,0.005170
1,201906,0.995294,0.004706
2,201907,0.994729,0.005271
3,201908,0.995839,0.004161
4,201909,0.995712,0.004288
5,201910,0.995435,0.004565
6,201911,0.994700,0.005300
7,201912,0.995749,0.004251
8,202001,0.996513,0.003487
9,202002,0.998742,0.001258


In [ ]:
df_subsampled = []

for mes, group in data.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    # print(f'Original proportion:{minority_proportion}')
    
    # voy a incrementar esa proporcion por 10
    new_proportion = minority_proportion * 10

    rus = RandomUnderSampler(sampling_strategy=new_proportion, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rearmar
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes

    df_subsampled.append(group_resampled)

# Mergear
data = pd.concat(df_subsampled, ignore_index=True)

data.shape

In [ ]:
# proporciones luego de hacer el ajuste

proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,201905,0.950838,0.049162
1,201906,0.955050,0.044950
2,201907,0.949931,0.050069
3,201908,0.960052,0.039948
4,201909,0.958878,0.041122
5,201910,0.956339,0.043661
6,201911,0.949661,0.050339
7,201912,0.959220,0.040780
8,202001,0.966304,0.033696
9,202002,0.987580,0.012420


In [13]:
output_file = 'competencia_03_fe_v02_undersampled_10_altbinaria.parquet'

data.to_parquet('/home/eanegrin/datasets/' + output_file, index=False)

In [14]:
!gsutil cp /home/eanegrin/datasets/competencia_03_fe_v02_undersampled_10_altbinaria.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_03_fe_v02_undersampled_10_altbinaria.parquet...
- [1 files][588.9 MiB/588.9 MiB]                                                
Operation completed over 1 objects/588.9 MiB.                                    
